<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# SQL Together Lab: Learning SQL Syntax


---




### Learning Objectives

- Sort results by column using `ORDER BY`
- Simplify your syntax using aliases (`AS`)
- Match patterns using `LIKE`
- Select distinct items using `DISTINCT`
- Aggregate values using `GROUP BY`
- Filter on aggregations using `HAVING`
- Apply `IF/THEN` logic using `CASE`
- Use `EXTRACT` to get date parts

### Lesson Guide
- [Install `psycopg2`](#install-psycopg2)
- [Connect to a Remote Database](#connect-to-remote)
- [Some Notes on Syntax](#syntax-notes)
- [ORDER BY](#order-by)
- [Alias `AS`](#alias-as)
- [LIKE](#like-operator)
- [DISTINCT](#distinct)
- [LIMIT](#limit)
- [GROUP BY](#group-by)
- [HAVING](#having)
- [CASE Statements](#case)
- [Working with Dates](#dates)
- [Additional Exercises](#additional-exercises)
- [Conclusion](#conclusion)
- [Additional Resources](#additional-resources)


<a id='install-psycopg2'></a>
## Install `psycopg2`

---



`conda install -c anaconda psycopg2`



<a id='connect-to-remote'></a>
## Connect to a Remote Database

---

In [3]:
import psycopg2
import pandas as pd

# DSN (data source name) format for database connections:  
# [protocol / database  name]://[username]:[password]@[hostname / ip]:[port]/[database name here]


# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database = 'northwind'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

<a id='order-by'></a>

## `ORDER BY`

---

    The `ORDER BY` keyword is used to sort a result set by one or more columns. It sorts records in ascending order by default. To sort the records in descending order, you can use the `DESC` keyword.

### SQL `ORDER BY` Syntax

```*.sql
SELECT column_name1, column_name2  
FROM table_name  
ORDER BY column_name1 ASC, column_name2 DESC;
``` 

### Exercise #1:

Select the `product_id`, `product_name`, `supplier_id`, and `unit_price` for all `products` with a `unit_price > 25`, ordered by `supplier_id` descending and then `unit_price` ascending.

In [5]:
query = """
SELECT product_id, product_name, supplier_id, unit_price
FROM products
WHERE unit_price > 25
ORDER BY supplier_id DESC, unit_price ASC
"""

df = pd.read_sql(query, con=conn)
df.head(10)

,product_id,product_name,supplier_id,unit_price
0,61,Sirop d'érable,29,28.5
1,62,Tarte au sucre,29,49.3
2,60,Camembert Pierrot,28,34.0
3,59,Raclette Courdavault,28,55.0
4,56,Gnocchi di nonna Alice,26,38.0
5,53,Perth Pasties,24,32.8
6,51,Manjimup Dried Apples,24,53.0
7,43,Ipoh Coffee,20,46.0
8,38,Côte de Blaye,18,263.5
9,37,Gravad lax,17,26.0


## <a id='alias-as'></a>
## Alias `AS`

---

SQL aliases are used to give a database table — or a column in a table — a temporary name. Aliases are often created for two purposes:
1. To make output column names more readable (substitute names). 
2. To make queries more concise (shorten query arguments).

### SQL Alias Syntax for Columns

```*.sql
SELECT column_name AS alias_name  
FROM table_name;
```

### SQL Alias Syntax for Tables

```*.sql
SELECT column_name(s)  
FROM table_name AS alias_name;
```



### Exercise #2

Select `supplier_id` and `company_name` from the `suppliers` table, aliasing these columns as `Supplier No.` and `Company Name`, respectively. Additionally, alias the `suppliers` table as `S`. Order by `Company Name` ascending.

**Hint:** Column names with spaces have to be enclosed by quotation marks.

In [15]:
query = '''
SELECT supplier_id AS "Supplier No.", company_name AS "Company Name"
FROM suppliers AS S
ORDER BY "Company Name" ASC
'''

df = pd.read_sql(query, con=conn)
df.head(10)

,Supplier No.,Company Name
0,18,Aux joyeux ecclésiastiques
1,16,Bigfoot Breweries
2,5,Cooperativa de Quesos 'Las Cabras'
3,27,Escargots Nouveaux
4,1,Exotic Liquids
5,14,Formaggi Fortini s.r.l.
6,29,Forêts d'érables
7,24,"G'day, Mate"
8,28,Gai pâturage
9,3,Grandma Kelly's Homestead


**Aliases can be useful when:**

- More than one table is involved in a query
- Functions are used in the query
- Column names are long and/or not very readable
- Two or more columns are combined together

<a id='like-operator'></a>
## SQL's `LIKE` Operator

---

The `LIKE` operator is used in a `WHERE` clause to search for a specific pattern within a column.


### SQL `LIKE` Syntax

```*.sql

SELECT column_name(s) 
FROM table_name  
WHERE column_name LIKE pattern;

```

> **Tip**: The `"%"` sign is used to define wildcards (missing letters) both before and after the pattern. Also, notice that PostgreSQL is case sensitive.

### Exercise #3

In descending order, select all products from the `products` table with a `product_name` that contains "ch." Alias this column as `Ch Products`. 

In [82]:
query = '''
SELECT product_name AS "Ch Products" 
FROM products 
WHERE product_name LIKE '%ch%' 
ORDER BY "Ch Products" 
DESC LIMIT 10
'''

pd.read_sql(query, con=conn)

,Ch Products
0,Schoggi Schokolade
1,Sasquatch Ale
2,Queso Manchego La Pastora
3,Pâté chinois
4,Gumbär Gummibärchen
5,Gnocchi di nonna Alice


### Exercise #4

In ascending order of `product_name`, select all products from the `suppliers` table with a `city` that starts with "S." Alias this column as `S Cities`. 

In [78]:
query = '''
SELECT A.product_name, B.city AS "S Cities"
FROM products A LEFT JOIN suppliers B
ON A.supplier_id = B.supplier_id
WHERE B.city LIKE 'S%'
'''

pd.read_sql(query, con=conn)

,product_name,S Cities
0,Guaraná Fantástica,Sao Paulo
1,Geitost,Sandvika
2,Inlagd Sill,Stockholm
3,Gravad lax,Stockholm
4,Singaporean Hokkien Fried Mee,Singapore
5,Ipoh Coffee,Singapore
6,Gula Malacca,Singapore
7,Manjimup Dried Apples,Sydney
8,Filo Mix,Sydney
9,Perth Pasties,Sydney


<a id='distinct'></a>
## The `DISTINCT` operator

---

The `SELECT DISTINCT` statement is used to return _only_ distinct (unique) values. In a table, a column may contain many duplicate values; sometimes you'll only want to list the unique ones.

### `SELECT DISTINCT` Syntax

```*.sql

SELECT DISTINCT column_name1, column_name2 
FROM table_name;

```

### Exercise #5

`SELECT DISTINCT` `supplier_id`, `product_name`, and `unit_price` from the `products` table, ordering by `unit_price` ascending (i.e., the cheapest product for each supplier).

In [80]:
query = '''
SELECT DISTINCT supplier_id, product_name, unit_price
FROM products
ORDER BY unit_price ASC
LIMIT 5
'''
pd.read_sql(query, con=conn)

,supplier_id,product_name,unit_price
0,15,Geitost,2.50
1,10,Guaraná Fantástica,4.50
2,6,Konbu,6.00
3,24,Filo Mix,7.00
4,25,Tourtière,7.45


<a id='limit'></a>

## The `LIMIT` operator

---

Sometimes, we may want to only retrieve a fixed number of records from a database. This is where the `LIMIT` operator comes in handy.


### `LIMIT` Syntax

```*.sql

SELECT column_name1, column_name2  
FROM table_name
LIMIT number_of_records;

```

## Sub-queries

---
A subquery is an SQL query nested inside a larger query.


### `sub-query` Syntax

```*.sql

SELECT sub.column_names()
FROM 
        (SELECT column_name(s)
        FROM table
        WHERE ...
        ORDER BY ...) as sub
        
ORDER BY ... 
LIMIT ...
       
```


**OR**

```*.sql

WITH sub as (SELECT column_name(s)
            FROM table
            WHERE ...
            ORDER BY ...)
            
SELECT sub.column_name(s)
FROM sub
ORDER BY ...
LIMIT ...
       
```

### Exercise #6

In ascending order, return the five highest-priced products according to `unit_price` that contain an "a" in the product name. Alias the column as `Top 5 A Products`.

_**Tip:** This one requires a sub-query!_

In [90]:
query = '''
With sub as (SELECT product_name as "Top 5 A Products", unit_price
            FROM products
            WHERE product_name LIKE '%a%'
            ORDER BY unit_price DESC
            LIMIT 5)
SELECT *
FROM sub
ORDER BY unit_price ASC
'''
pd.read_sql(query, con=conn)

,Top 5 A Products,unit_price
0,Raclette Courdavault,55.00
1,Carnarvon Tigers,62.50
2,Sir Rodney's Marmalade,81.00
3,Thüringer Rostbratwurst,123.79
4,Côte de Blaye,263.50


_**Tip:** If you are finding this one a bit tricky to execute in one query, check out [SQL Subqueries](https://www.tutorialspoint.com/sql/sql-sub-queries.htm)._

<a id='group-by'></a>
## `GROUP BY` Operator

---

A table may contain several records that have a common key. 

The `GROUP BY` statement is used in conjunction with aggregate functions to group a result set by one or more columns. For example, we may want to know the total number of items purchased in each order.

### `GROUP BY` Syntax

```*.sql
SELECT column_name, aggregate_function(column_name)  
FROM table_name  
WHERE column_name operator value  
GROUP BY column_name;
```

The aggregate functions you can use with `GROUP BY` are:
- **`COUNT`**
- **`MIN`**
- **`MAX`**
- **`SUM`**
- **`AVG`**

### Exercise #7

From the `order_details` table, show the count of orders per `order_id`, as well as the `SUM` of the revenue (`unit_price * quantity`). Order by revenue.

In [99]:
query = '''
SELECT order_id, COUNT(product_id), SUM(unit_price*quantity)
FROM order_details
GROUP BY order_id
LIMIT 5
'''
pd.read_sql(query, con=conn)

,order_id,count,sum
0,10248,3,439.999998
1,10249,2,1863.400064
2,10250,3,1813.000040
3,10251,3,670.799986
4,10252,3,3730.000153


<a id='having'></a>
## The `HAVING` operator

---

The `HAVING` clause was added to SQL because the `WHERE` keyword could not be used with aggregate functions. `HAVING` allows us to apply a filter while querying with them. For example, if we only wanted to show companies that had revenues greater than $10,000 (as calculated by an aggregate function).

### `HAVING` Syntax

``` *.sql

SELECT column_name, aggregate_function(column_name)
FROM table_name
WHERE column_name operator value
GROUP BY column_name
HAVING aggregate_function(column_name) operator value;

```

### Exercise #8

Show the revenue of all orders with more than one item.

In [107]:
query = '''
SELECT order_id, COUNT(product_id), SUM(unit_price*quantity)
FROM order_details
GROUP BY order_id
HAVING COUNT(product_id) > 1
LIMIT 5
'''
pd.read_sql(query, con=conn)

,order_id,count,sum
0,10248,3,439.999998
1,10249,2,1863.400064
2,10250,3,1813.000040
3,10251,3,670.799986
4,10252,3,3730.000153


<a id='case'></a>
## `CASE` statements

---

The `CASE` statement is SQL’s way of applying `IF/THEN` logic. The `CASE` statement is followed by at least one pair of `WHEN` and `THEN` statements. It must end with an `END` statement. The `ELSE` statement is optional and provides a way to capture values not specified in the `WHEN/THEN` statements.

### `CASE` Syntax

Generic form: 

```*.sql
SELECT 
    CASE 
        WHEN column_name operator value THEN 'string value1'
        WHEN column_name operator value THEN 'string value2'
        ELSE 'string value3' 
     END 
     AS 'alias'         
FROM table_name
```

Or, when testing values in one column only when we're testing for equality: 


```*.sql
SELECT 
    CASE column_name 
        WHEN value THEN 'string value1'
        WHEN value THEN 'string value2'
        ELSE 'string value3' 
    END 
    AS 'alias'         
FROM table_name
```


### A Pseudocode Example

```*.sql
SELECT name,
    CASE 
        WHEN age < 1 THEN 'infant'
        WHEN age < 2 THEN 'toddler'
        WHEN age < 5 THEN 'child'
        ELSE 'old as dirt' 
    END 
    AS 'Persons Age'
```

### Exercise #9

Select `company_name`, `city`, and `country` from the `suppliers` table. Add a new column, `domestic_foreign`, which contains a value of "domestic" if the supplier is from the United States and "foreign" if not.

In [125]:
query = '''
SELECT company_name, city, country,
    CASE country
        WHEN 'USA' THEN 'domestic'
        ELSE 'foreign'
    END
    AS "domestic_foreign"
FROM suppliers
LIMIT 5
'''

pd.read_sql(query, con=conn)

,company_name,city,country,domestic_foreign
0,Exotic Liquids,London,UK,foreign
1,New Orleans Cajun Delights,New Orleans,USA,domestic
2,Grandma Kelly's Homestead,Ann Arbor,USA,domestic
3,Tokyo Traders,Tokyo,Japan,foreign
4,Cooperativa de Quesos 'Las Cabras',Oviedo,Spain,foreign


<a id='dates'></a>
## Working With Dates

---

Take some time to look over the [PostgreSQL date documentation](https://www.postgresql.org/docs/8.1/static/functions-datetime.html).

### Extracting Date Parts From a Date Object
```*.sql
SELECT my_date,
       EXTRACT('year'   FROM my_date) AS year,
       EXTRACT('month'  FROM my_date) AS month,
       EXTRACT('day'    FROM my_date) AS day,
       EXTRACT('hour'   FROM my_date) AS hour,
       EXTRACT('minute' FROM my_date) AS minute,
       EXTRACT('second' FROM my_date) AS second,
       EXTRACT('decade' FROM my_date) AS decade,
       EXTRACT('dow'    FROM my_date) AS day_of_week
FROM table_name
```

Alternatively, you can use postgres' `date_part` function:

```SQL
SELECT date_part('year', my_date) AS year 
FROM table_name
```

### Exercise #10

Select `order_date` and `freight` from the `orders` table, along with three new columns for `year`, `month`, and `day`. Make sure these are [_**cast**_ as integers, not floats](http://www.postgresqltutorial.com/postgresql-cast/).

After extracting the dates as integers, pull out the `year`, `month`, and `SUM` of `freight`, aliased as `freight_per_month`, grouping by the `year` and `month`, but only where the `freight_per_month` is greater than 5,000.

Order this DataFrame by `year` and `month` descending.

In [170]:
query = '''
With Sub as (SELECT order_date, freight,
                EXTRACT('year' FROM order_date) AS year,
                EXTRACT('month' FROM order_date) AS month,
                EXTRACT('day' FROM order_date) AS day
            FROM orders)
SELECT CAST(year AS INTEGER), CAST(month AS INTEGER), SUM(freight) AS "freight_per_month"
FROM Sub
GROUP BY year, month
HAVING SUM(freight) > 5000
'''
pd.read_sql(query, con=conn)

,year,month,freight_per_month
0,1998,4,6393.5703
1,1998,1,5463.4404
2,1998,3,5379.0195


In [197]:
pd.read_sql('''
SELECT table_name, data_type, column_name
FROM information_schema.columns
WHERE table_name = 'orders'
''', con=conn)

,table_name,data_type,column_name
0,orders,smallint,order_id
1,orders,character,customer_id
2,orders,smallint,employee_id
3,orders,date,order_date
4,orders,date,required_date
5,orders,date,shipped_date
6,orders,smallint,ship_via
7,orders,real,freight
8,orders,character varying,ship_name
9,orders,character varying,ship_address


### Exercise #11

From the `orders` table, find the average number of days it took to ship a package per `ship_country`. Only include orders that have a `shipped_date`, and only show the top five results.

In [179]:
pd.read_sql('''SELECT * FROM orders LIMIT 5''', con=conn)

,order_id,customer_id,employee_id,order_date,required_date,shipped_date,ship_via,freight,ship_name,ship_address,ship_city,ship_region,ship_postal_code,ship_country
0,10248,VINET,5,1996-07-04,1996-08-01,1996-07-16,3,32.38,Vins et alcools Chevalier,59 rue de l'Abbaye,Reims,None,51100,France
1,10249,TOMSP,6,1996-07-05,1996-08-16,1996-07-10,1,11.61,Toms Spezialitäten,Luisenstr. 48,Münster,None,44087,Germany
2,10250,HANAR,4,1996-07-08,1996-08-05,1996-07-12,2,65.83,Hanari Carnes,"Rua do Paço, 67",Rio de Janeiro,RJ,05454-876,Brazil
3,10251,VICTE,3,1996-07-08,1996-08-05,1996-07-15,1,41.34,Victuailles en stock,"2, rue du Commerce",Lyon,None,69004,France
4,10252,SUPRD,4,1996-07-09,1996-08-06,1996-07-11,2,51.30,Suprêmes délices,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium


### Exercise #12

In the `orders` table, find the top five countries by average `freight` cost of products shipped in 1998.

### Exercise #13

From the `employees` table, find the two women who were hired the most recently. Exclude entries where gender is ambiguous.  
_**Tip:** You may want to investigate the "title_of_courtesy" column._

### Exercise #14

Split products from the `products` table into three price categories:
- **Cheap**: Less than \$10.
- **Fair**: \$ 10 to 50.
- **Expensive**: Greater than \$50.

Return the count-per-product price categories, along with the `MIN`, `MAX`, and `AVG`. 

<a id='conclusion'></a>
## Conclusion

---

In this lesson, we've learned many new commands for making powerful SQL queries.

In particular, we learned how to:

- Sort results by column using `ORDER BY`
- Simplify our syntax using aliases
- Match patterns using `LIKE`
- Select distinct items using `DISTINCT`
- Aggregate values using `GROUP BY`
- Filter aggregations using `HAVING`
- Apply `IF/THEN` logic using `CASE`
- Use `EXTRACT` to get date part

**Can you think of a few more business cases where these capabilities would be useful?**

<a id='additional-resources'></a>
## Additional Resources

---

- [PostgreSQL Documenation](https://www.postgresql.org/docs/)
- [Mode Analytics Tutorial](https://community.modeanalytics.com/sql/tutorial/introduction-to-sql/)